In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
df = pd.read_csv('train_raw.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_22024\2013681907.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train_raw.csv')


In [9]:
df.head(20)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,January,23,Scientist,19114.12,1824.843333,3,4,3,4,3,...,_,809.98,26.822620,221.0,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
1,February,23,Scientist,19114.12,NaN,3,4,3,4,1,...,Good,809.98,31.944960,NaN,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good
2,March,500,Scientist,19114.12,NaN,3,4,3,4,3,...,Good,809.98,28.609352,223.0,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good
3,April,23,Scientist,19114.12,NaN,3,4,3,4,5,...,Good,809.98,31.377862,224.0,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good
4,May,23,Scientist,19114.12,1824.843333,3,4,3,4,6,...,Good,809.98,24.797347,225.0,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good
5,June,23,Scientist,19114.12,NaN,3,4,3,4,8,...,Good,809.98,27.262259,226.0,No,49.574949,62.430172,!@9#%8,340.479212,Good
6,July,23,Scientist,19114.12,1824.843333,3,4,3,4,3,...,Good,809.98,22.537593,227.0,No,49.574949,178.344067,Low_spent_Small_value_payments,244.565317,Good
7,August,23,Scientist,19114.12,1824.843333,3,4,3,4,3,...,Good,809.98,23.933795,NaN,No,49.574949,24.785217,High_spent_Medium_value_payments,358.124168,Standard
8,January,28,_______,34847.84,3037.986667,2,4,6,1,3,...,Good,605.03,24.464031,267.0,No,18.816215,104.291825,Low_spent_Small_value_payments,470.690627,Standard
9,February,28,Teacher,34847.84,3037.986667,2,4,6,1,7,...,Good,605.03,38.550848,268.0,No,18.816215,40.391238,High_spent_Large_value_payments,484.591214,Good


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [5]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

print("Columns with missing values:")
print(missing_values)

Columns with missing values:
Name                        9985
Monthly_Inhand_Salary      15002
Type_of_Loan               11408
Num_of_Delayed_Payment      7002
Num_Credit_Inquiries        1965
Credit_History_Age          9030
Amount_invested_monthly     4479
Monthly_Balance             1200
dtype: int64


In [6]:
max_missing_threshold = 20000  
max_unique_threshold = 10000    

extra_columns_to_remove = ['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan']
df = df.drop(columns=extra_columns_to_remove, errors='ignore')

high_null_columns = df.columns[df.isnull().sum() > max_missing_threshold]

high_unique_columns = [col for col in df.select_dtypes(include=['object', 'category'])
                       if df[col].nunique() > max_unique_threshold]

In [7]:
num_cols = [
    "Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card",
    "Interest_Rate", "Num_of_Loan", "Delay_from_due_date", "Num_of_Delayed_Payment",
    "Changed_Credit_Limit", "Num_Credit_Inquiries", "Outstanding_Debt",
    "Credit_Utilization_Ratio", "Total_EMI_per_month", "Amount_invested_monthly",
    "Monthly_Balance", "Credit_History_Age"
]



def clean_numeric_columns(df, num_cols):
    for col in num_cols:
        df[col] = df[col].astype(str).str.replace(r"[^\d.]", "", regex=True)  
        df[col] = pd.to_numeric(df[col], errors='coerce') 
    return df

# Clean numerical columns
df = clean_numeric_columns(df, num_cols)
df.head(10)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,January,23,Scientist,19114.12,1824.843333,3,4,3,4,3,...,_,809.98,26.822620,221.0,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
1,February,23,Scientist,19114.12,NaN,3,4,3,4,1,...,Good,809.98,31.944960,NaN,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good
2,March,500,Scientist,19114.12,NaN,3,4,3,4,3,...,Good,809.98,28.609352,223.0,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good
3,April,23,Scientist,19114.12,NaN,3,4,3,4,5,...,Good,809.98,31.377862,224.0,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good
4,May,23,Scientist,19114.12,1824.843333,3,4,3,4,6,...,Good,809.98,24.797347,225.0,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good
5,June,23,Scientist,19114.12,NaN,3,4,3,4,8,...,Good,809.98,27.262259,226.0,No,49.574949,62.430172,!@9#%8,340.479212,Good
6,July,23,Scientist,19114.12,1824.843333,3,4,3,4,3,...,Good,809.98,22.537593,227.0,No,49.574949,178.344067,Low_spent_Small_value_payments,244.565317,Good
7,August,23,Scientist,19114.12,1824.843333,3,4,3,4,3,...,Good,809.98,23.933795,NaN,No,49.574949,24.785217,High_spent_Medium_value_payments,358.124168,Standard
8,January,28,_______,34847.84,3037.986667,2,4,6,1,3,...,Good,605.03,24.464031,267.0,No,18.816215,104.291825,Low_spent_Small_value_payments,470.690627,Standard
9,February,28,Teacher,34847.84,3037.986667,2,4,6,1,7,...,Good,605.03,38.550848,268.0,No,18.816215,40.391238,High_spent_Large_value_payments,484.591214,Good


In [8]:




# Impute
knn_imputer = KNNImputer(n_neighbors=5)

df[num_cols] = knn_imputer.fit_transform(df[num_cols])

KeyboardInterrupt: 

In [ ]:
print(df.isnull().sum())


Month                       0
Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64


In [ ]:
print(df.duplicated().sum())

0


In [ ]:
df_encoded = df.copy()
for col in df_encoded.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))


X = df_encoded.drop(columns=['Credit_Score'])
y = df_encoded['Credit_Score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

k = min(10, X_train.shape[1])

# Apply SelectKBest
select_k_best = SelectKBest(score_func=chi2, k=k)
X_train_k_best = select_k_best.fit_transform(X_train, y_train)

# Get selected feature names
selected_features = X_train.columns[select_k_best.get_support()]

# Print selected features
print("Selected features:", selected_features.tolist())

# Create DataFrame for feature scores
feature_scores = pd.DataFrame({'Feature': X_train.columns, 'Score': select_k_best.scores_})
feature_scores = feature_scores.sort_values(by="Score", ascending=False)

# Print feature scores
print("\nFeature Scores:")
print(feature_scores)



Selected features: ['Annual_Income', 'Monthly_Inhand_Salary', 'Interest_Rate', 'Delay_from_due_date', 'Changed_Credit_Limit', 'Outstanding_Debt', 'Credit_History_Age', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Monthly_Balance']

Feature Scores:
                     Feature         Score
21           Monthly_Balance  1.687044e+27
3              Annual_Income  1.071016e+08
14          Outstanding_Debt  1.000486e+07
4      Monthly_Inhand_Salary  7.411940e+06
16        Credit_History_Age  1.026920e+06
18       Total_EMI_per_month  1.933281e+05
9        Delay_from_due_date  1.360119e+05
19   Amount_invested_monthly  1.005454e+05
7              Interest_Rate  1.773547e+04
11      Changed_Credit_Limit  1.521642e+04
10    Num_of_Delayed_Payment  1.424371e+04
12      Num_Credit_Inquiries  1.229941e+04
1                        Age  1.154198e+04
5          Num_Bank_Accounts  3.231692e+03
8                Num_of_Loan  2.503742e+03
17     Payment_of_Min_Amount  2.145090e+03
13             

In [ ]:
le = LabelEncoder()
le.fit(df['Credit_Score']) 
print("\nLabel Encoding Mapping:")
for idx, class_name in enumerate(le.classes_):
    print(f"{class_name} → {idx}")

selected_features = X_train.columns[select_k_best.get_support()]
X_train_selected = pd.DataFrame(X_train_k_best, columns=selected_features, index=X_train.index)
X_train_selected['Credit_Score'] = y_train

# Split 10% of the selected training data as unseen data
selected_train, selected_unseen = train_test_split(X_train_selected, test_size=0.1, random_state=42)


selected_train.to_csv("cleaned_train.csv", index=False)
selected_unseen.to_csv("cleaned_unseen.csv", index=False)


Label Encoding Mapping:
Good → 0
Poor → 1
Standard → 2
